In [ ]:
from operator import sub
import os
import numpy as np
import matplotlib.pyplot as plt
from argparse import Namespace
from Utils.PySqlite import *
from myconfig import *
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [ ]:
methods = ['mondrian', 'classic_mondrian', 'topdown']  # ['cluster', 'datafly']
dataset = ['adult', 'cahousing', 'cmc', 'mgm', 'informs']  # italia
metrics = ['ncp', 'cav', 'dm']
lcolors = ['orange', 'deepskyblue', 'limegreen', 'magenta']
metric_names = [
    'Normalized\nCertainty\n(lower is better)',
    'Average\nEquivalence\n(lower is better)',
    'Discernibility\nMetric\n(lower is better)']
label_x = dataset
label_y = metric_names
figname = 'data/result/plot_eval'


In [ ]:
conn = CreateConnection(DATABASE_PATH)
with conn:
    df = Find(conn)
df.head()

In [ ]:
result = np.genfromtxt('metric_result', names=[
                       'data', 'method', 'k', 'time','lrp', 'nrp', 'drp','cavc'], dtype=None, encoding=None)
print(result)
raise
fig, axis = plt.subplots(
    nrows=len(metrics), ncols=len(dataset), figsize=(35, 30))

for row, metric in enumerate(metrics):
    for col, data in enumerate(dataset):
        data = data.encode('utf-8')
        sub_data = result[(data == result['data'])]
        for i, method in enumerate(methods):
            method = method.encode('utf-8')
            sub = sub_data[(method == sub_data['method'])]
            axis[row, col].plot(
                sub['k'], sub[metric], color=lcolors[i], label=sub['method'][0].decode('utf-8'))

labels_handles = {
    label: handle for ax in fig.axes for handle, label in zip(*ax.get_legend_handles_labels())
}

fig.legend(
    labels_handles.values(),
    labels_handles.keys(),
    loc='upper center',
    fontsize=30,
    ncol=len(labels_handles.values()))

for ax, col in zip(axis[0], label_x):
    ax.set_title(col.upper(), size=20)

for ax in axis[-1]:
    ax.set_xlabel('k', size=20)

for ax, row in zip(axis[:, 0], label_y):
    ax.set_ylabel(row, size=30)
    ax.get_yaxis().set_label_coords(-0.2, 0.5)

plt.subplots_adjust(0.075, 0.05, 0.97, 0.95, 0.2, 0.25)
plt.savefig(figname)
plt.show()
